In [27]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [28]:
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize 

plt.rcParams['figure.figsize'] = (13, 7)

In [29]:


#take a look on the dataset
df = pd.read_csv("random_data.csv")
df.head(5)



,Name,Skill,Education,Experience
0,Susan Moyer,"Accounting, Economics, Business Awareness, Dat...",Bachelors degree in Finance,17 years
1,Linda Mayo,"Medicine Dosage Calculation, Patient Care, Com...",Bachelors degree in Nursing,17 years
2,Donald Robinson,"Lesson Planning, Teaching, Education Psycholog...",Bachelors degree in Teaching,5 years
3,Katherine Lane,"Problem Solving, Backend Development, JavaScri...",Bachelors degree in Web Development,2 years
4,Ernest Reed,"Nutrition Therapy, Diet plan, Molecular Biolog...",Bachelors degree in Nutrition,20 years


In [30]:


#showing info for each featrue
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        6000 non-null   object
 1   Skill       6000 non-null   object
 2   Education   6000 non-null   object
 3   Experience  6000 non-null   object
dtypes: object(4)
memory usage: 187.6+ KB


In [31]:


df.head(1)



,Name,Skill,Education,Experience
0,Susan Moyer,"Accounting, Economics, Business Awareness, Dat...",Bachelors degree in Finance,17 years


In [32]:


#now checking the NaN values
df.isnull().sum()



Name          0
Skill         0
Education     0
Experience    0
dtype: int64

In [33]:
df['Skill'] = df['Skill'].fillna("Unknown")
df['Education'] = df['Education'].fillna("Unknown")
df['Experience'] = df['Experience'].fillna("Unknown")
df = df.reset_index( drop=True)


In [34]:


df.head(5)



,Name,Skill,Education,Experience
0,Susan Moyer,"Accounting, Economics, Business Awareness, Dat...",Bachelors degree in Finance,17 years
1,Linda Mayo,"Medicine Dosage Calculation, Patient Care, Com...",Bachelors degree in Nursing,17 years
2,Donald Robinson,"Lesson Planning, Teaching, Education Psycholog...",Bachelors degree in Teaching,5 years
3,Katherine Lane,"Problem Solving, Backend Development, JavaScri...",Bachelors degree in Web Development,2 years
4,Ernest Reed,"Nutrition Therapy, Diet plan, Molecular Biolog...",Bachelors degree in Nutrition,20 years


In [35]:
overall_infos = []
for i in range(0, df.shape[0]):
    overall_infos.append(df['Name'][i]+' '+df['Skill'][i]+' '+df['Education'][i]+' '+df['Experience'][i])
df['overall_infos'] = overall_infos

In [36]:
df_new = df[['overall_infos']]
df_new.head(2)

,overall_infos
0,"Susan Moyer Accounting, Economics, Business Aw..."
1,"Linda Mayo Medicine Dosage Calculation, Patien..."


In [37]:
#Stopwords help us to get rid of unwanted words like: a, an, are, is, ...
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [38]:
def text_preprocessing(column):
    #make all words with lower letters
    column = column.str.lower()
    #getting rid of any punctution
    column = column.str.replace('http\S+|www.\S+|@|%|:|,|', '', case=False)
    #spliting each sentence to words to apply previous funtions on them 
    word_tokens = column.str.split()
    keywords = word_tokens.apply(lambda x: [item for item in x if item not in stop])
    #assemble words of each sentence again and assign them in new column
    for i in range(len(keywords)):
        keywords[i] = " ".join(keywords[i])
        column = keywords

    return column

In [39]:


df_new.loc[:, 'cleaned_infos'] = text_preprocessing(df_new['overall_infos'])




/tmp/ipykernel_23806/3345913491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.loc[:, 'cleaned_infos'] = text_preprocessing(df_new['overall_infos'])


In [40]:


df['overall_infos']



0       Susan Moyer Accounting, Economics, Business Aw...
1       Linda Mayo Medicine Dosage Calculation, Patien...
2       Donald Robinson Lesson Planning, Teaching, Edu...
3       Katherine Lane Problem Solving, Backend Develo...
4       Ernest Reed Nutrition Therapy, Diet plan, Mole...
                              ...                        
5995    Tyler Shah Adobe Creative Suite, Typography, L...
5996    Charlotte Howard JavaScript, Problem Solving, ...
5997    Marc Smith Database Administration, Programmin...
5998    Michelle Mcpherson Business Awareness, Account...
5999    Brittany Friedman Web design, Data analysis, S...
Name: overall_infos, Length: 6000, dtype: object

In [41]:
df_new['cleaned_infos']

0       susan moyer accounting, economics, business aw...
1       linda mayo medicine dosage calculation, patien...
2       donald robinson lesson planning, teaching, edu...
3       katherine lane problem solving, backend develo...
4       ernest reed nutrition therapy, diet plan, mole...
                              ...                        
5995    tyler shah adobe creative suite, typography, l...
5996    charlotte howard javascript, problem solving, ...
5997    marc smith database administration, programmin...
5998    michelle mcpherson business awareness, account...
5999    brittany friedman web design, data analysis, s...
Name: cleaned_infos, Length: 6000, dtype: object

In [42]:
from sklearn.model_selection import train_test_split  # Importing train_test_split
# Define stopwords
stop = stopwords.words('english')

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(df_new, test_size=0.2, random_state=42)

# Assuming df_new['cleaned_infos'] contains the resumes data
train_resumes = train_data['cleaned_infos']
test_resumes = test_data['cleaned_infos']

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

CV = CountVectorizer()
converted_metrix = CV.fit_transform(df_new['cleaned_infos'])

In [44]:


cosine_similarity = cosine_similarity(converted_metrix)



In [45]:
cosine_similarity

array([[1.        , 0.25048972, 0.18786729, ..., 0.36514837, 0.82807867,
        0.31311215],
       [0.25048972, 1.        , 0.23529412, ..., 0.17149859, 0.19446112,
        0.23529412],
       [0.18786729, 0.23529412, 1.        , ..., 0.17149859, 0.19446112,
        0.23529412],
       ...,
       [0.36514837, 0.17149859, 0.17149859, ..., 1.        , 0.37796447,
        0.34299717],
       [0.82807867, 0.19446112, 0.19446112, ..., 0.37796447, 1.        ,
        0.32410186],
       [0.31311215, 0.23529412, 0.23529412, ..., 0.34299717, 0.32410186,
        1.        ]])

In [46]:
#finding the correct name of a skill
df[df['Skill'].str.contains('Finance')]

,Name,Skill,Education,Experience,overall_infos


In [47]:
!pip install torch

import torch
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
import re
import string
import nltk
from nltk.corpus import stopwords

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    
    # Remove special characters and non-alphanumeric characters (excluding spaces)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    
    # Remove punctuation
    tokens = [word for word in tokens if word not in string.punctuation]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Reconstruct the text
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

# Example job descriptions
job_descriptions = [
    '''


About the Role




'''
]

# Apply the cleaning function to each job description
cleaned_descriptions = [clean_text(desc) for desc in job_descriptions]

# Display the cleaned job descriptions
for original, cleaned in zip(job_descriptions, cleaned_descriptions):
    print(f"Original:\n{original}\nCleaned:\n{cleaned}\n")


Original:



About the Role





Cleaned:
role



In [49]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel

# Initialize the DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define batch size
batch_size = 8  # Adjust according to your computational resources

# Define job descriptions
job_descriptions = [
    '''
123





    '''
]

# Tokenize and embed job descriptions
job_description_embeddings = []
for i in range(0, len(job_descriptions), batch_size):
    batch = job_descriptions[i:i+batch_size]
    tokens = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
    tokens.to(device)
    with torch.no_grad():
        output = model(**tokens)
    embeddings = output.last_hidden_state.mean(dim=1).cpu().numpy()
    job_description_embeddings.extend(embeddings)

# Assuming df_new['cleaned_infos'][4] contains a list of resumes
resumes = df_new['cleaned_infos'][4]

# Tokenize and embed resumes
resume_embeddings = []
for i in range(0, len(resumes), batch_size):
    batch = resumes[i:i+batch_size]
    tokens = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
    tokens.to(device)
    with torch.no_grad():
        output = model(**tokens)
    embeddings = output.last_hidden_state.mean(dim=1).cpu().numpy()
    resume_embeddings.extend(embeddings)

# Calculate similarity scores
similarity_scores = []
for job_embedding in job_description_embeddings:
    job_embedding = torch.tensor(job_embedding).to(device)
    job_embedding = job_embedding.unsqueeze(0)  # Add batch dimension
    for resume_embedding in resume_embeddings:
        resume_embedding = torch.tensor(resume_embedding).to(device)
        resume_embedding = resume_embedding.unsqueeze(0)  # Add batch dimension
        with torch.no_grad():
            score = torch.cosine_similarity(job_embedding, resume_embedding)
        similarity_scores.append(score.item())

# Define threshold for qualifying candidates
threshold = 0.5  # Adjust as needed

# Filter candidates based on similarity scores
qualified_candidates = [resume for resume, score in zip(resumes, similarity_scores) if score > threshold]


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [50]:
# Calculate cosine similarity between job descriptions and resumes
similarity_scores = cosine_similarity(job_description_embeddings, resume_embeddings)
similarity_scores

array([[0.7621418 , 0.59665704, 0.65583736, 0.67597055, 0.7065439 ,
        0.70006585, 0.5853081 , 0.60756356, 0.81090474, 0.5913018 ,
        0.63469505, 0.69136345, 0.6649379 , 0.6726779 , 0.7580738 ,
        0.76261973]], dtype=float32)

In [51]:
# Rank candidates for each job description based on similarity scores
num_top_candidates = 5
top_candidates = []

for i, job_description in enumerate(cleaned_descriptions):
    candidates_with_scores = list(enumerate(similarity_scores[i]))
    candidates_with_scores.sort(key=lambda x: x[1], reverse=True)
    top_candidates_for_job = candidates_with_scores[:num_top_candidates]
    top_candidates.append(top_candidates_for_job)

# Print the top candidates for each job description
for i, job_description in enumerate(cleaned_descriptions):
    print(f"Top candidates for JD {i+1}")
    for candidate_index, score in top_candidates[i]:
        print(f"  Candidate {candidate_index + 1} - Similarity Score: {score:.4f}")
        # Print additional information if needed
        # print(f"  Resume: {resumes[candidate_index]}")
    print()

Top candidates for JD 1
  Candidate 9 - Similarity Score: 0.8109
  Candidate 16 - Similarity Score: 0.7626
  Candidate 1 - Similarity Score: 0.7621
  Candidate 15 - Similarity Score: 0.7581
  Candidate 5 - Similarity Score: 0.7065

